!There is a bug in this, a mismatch between number of test_corpus and number of test_labels

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import reuters, stopwords
from nltk.tokenize import word_tokenize
import re

In [85]:
# get reuters dataset
nltk.download('reuters')


[nltk_data] Downloading package reuters to
[nltk_data]     /Users/leviathan/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [86]:
# extract list of documents for each category/label
categories = ['gas', 'wheat', 'cocoa', 'sugar', 'coffee', 'gold', 'silver', 'platinum', 'palladium']
doc_list = [fileid for category in categories for fileid in reuters.fileids(category)]

In [88]:

# get preset training and test documents from reuters dataset
train_doc = [doc for doc in doc_list if 'training' in doc]
test_doc = [doc for doc in doc_list if 'test' in doc]

In [89]:
# we join the words in the document to get the training and test corpus
train_corpus = [" ".join(reuters.words(doc)) for doc in train_doc]
test_corpus = [" ".join(reuters.words(doc)) for doc in test_doc]

**Cleaning the data**

In [ ]:
# remove stopwords
stop_words = stopwords.words('english')
test_corpus = [word for word in test_corpus if word not in stop_words]
train_corpus = [word for word in train_corpus if word not in stop_words]

In [ ]:
# lowercase the entire corpus
test_corpus = [word.lower() for word in test_corpus]
train_corpus = [word.lower() for word in train_corpus]
# remove punctuation using regular expressions
# @note not a great way to do it becuase it leave a dangling apostrophe
test_corpus = [re.sub(r'[^\w\s]', '', word) for word in test_corpus]
train_corpus = [re.sub(r'[^\w\s]', '', word) for word in train_corpus]

**Setting up the model**

In [ ]:
# Vectorize documents
vectorizer = CountVectorizer()

In [ ]:
bow_train = vectorizer.fit_transform(train_corpus)
bow_test = vectorizer.transform(test_corpus)

In [91]:
# !there is a bug here
# ensure that all labels in the test set also appear in the training set
train_categories = [reuters.categories(doc)[0] for doc in train_doc]
test_categories = [reuters.categories(doc)[0] for doc in test_doc]

missing_categories = set(test_categories) - set(train_categories)

if missing_categories:
    print('Warning: The following categories appear in the test set but not in the training set: ', missing_categories)

# add missing label to the test_labels
test_labels = [label if label in train_categories else 'missing' for label in test_categories]


In [ ]:
test_doc = [doc for doc in test_doc if reuters.categories(doc)[0] not in missing_categories]
test_categories = [reuters.categories(doc)[0] for doc in test_doc]


In [ ]:

encoder = LabelEncoder()
train_labels = encoder.fit_transform(train_categories)
test_labels = encoder.transform(test_categories)

In [ ]:

classifier = LogisticRegression()
classifier.fit(bow_train, train_labels)

In [ ]:

test_predictions = classifier.predict(bow_test)
print("Test Accuracy: ", accuracy_score(test_labels, test_predictions))
print(classification_report(test_labels, test_predictions, target_names=encoder.classes_))